# Download data and process interferogram stack
George Brencher

## Create and configure environment

In [1]:
import os
if not os.path.exists('/srv/conda/envs/isce2'):
    !mamba create -n isce2 --file https://github.com/UW-GDA/Friendly-InSAR-time-series/blob/main/conda-linux-64.lock
    !conda activate isce2
    !python -m ipykernel install --user --name=isce2

In [ ]:
# Note--if processing on headless server, need to run the following for the stack processor to work: 
# pip install opencv-python-headless

Make sure kernel is set to 'isce2' or 'Python [conda env:isce2]'

'topStack' is a Sentinel 1 interferogram stack processing workflow which has been contributed to isce. To get access to topsStack, which does not come with the conda install of isce, we need to clone isce2 from github. 

In [2]:
#!git clone https://github.com/isce-framework/isce2.git

In [3]:
import isce

# Set Environment variables so that you can call ISCE from the command line
os.environ['ISCE_HOME'] = os.path.dirname(isce.__file__)
os.environ['ISCE_ROOT'] = os.path.dirname(os.environ['ISCE_HOME'])
os.environ['PATH']+='{ISCE_HOME}/bin:{ISCE_HOME}/applications'.format(**os.environ)

# Set path to topsStack workflow in isce source directory
CWD = os.getcwd()
os.environ['PATH']+=f':{CWD}/isce2/contrib/stack/topsStack'
print(os.environ['PATH'])

This is the Open Source version of ISCE.
Some of the workflows depend on a separate licensed package.
To obtain the licensed package, please make a request for ISCE
through the website: https://download.jpl.nasa.gov/ops/request/index.cfm.
Alternatively, if you are a member, or can become a member of WinSAR
you may be able to obtain access to a version of the licensed sofware at
https://winsar.unavco.org/software/isce
/srv/conda/envs/isce2/bin:/srv/conda/condabin:/srv/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin/srv/conda/envs/isce2/lib/python3.9/site-packages/isce/bin:/srv/conda/envs/isce2/lib/python3.9/site-packages/isce/applications:/home/jovyan/Friendly-InSAR-time-series/isce2/contrib/stack/topsStack


In [4]:
# Make sure stack processing scripts are in path
!which stackSentinel.py

/home/jovyan/Friendly-InSAR-time-series/isce2/contrib/stack/topsStack/stackSentinel.py


In [38]:
# Do downloads and processing in a temporary folder to avoid clogging the disk
os.chdir('/tmp')
os.makedirs('orbital')
os.makedirs('SLC')
os.makedirs('DEM')
os.makedirs('AUX')

FileExistsError: [Errno 17] File exists: 'orbital'

## Download scenes

In [6]:
#check disk space prior to downloading SLCs
!df -h ~

Filesystem      Size  Used Avail Use% Mounted on
/dev/sdd         15G  206M   15G   2% /home/jovyan


In [32]:
os.chdir('/tmp/SLC')

In [7]:
import getpass
import asf_search as asf
import logging

# Update this with your NASA Earthdata login to download SLC data
EARTHDATA_LOGIN = "qbrencherUW"
EARTHDATA_PASSWORD = getpass.getpass()

# prevent DEBUG messages
logging.getLogger("urllib3").setLevel(logging.WARNING)

try:
    user_pass_session = asf.ASFSession().auth_with_creds(EARTHDATA_LOGIN, EARTHDATA_PASSWORD)
except asf.ASFAuthenticationError as e:
    print(f'Auth failed: {e}')
else:
    print('Success!')

 ········


Success!


In [8]:
# Create a persistent .netrc file for downloading NASA datasets (ISCE retrieves SRTM by default)
# https://wiki.earthdata.nasa.gov/display/EL/How+To+Access+Data+With+cURL+And+Wget
!echo "machine urs.earthdata.nasa.gov login {EARTHDATA_LOGIN} password {EARTHDATA_PASSWORD}" > ~/.netrc
!chmod 0600 ~/.netrc

In [9]:
# Set area of interest with two corner points of a box [lat, lon]

# southwestern corner
lower_left = [27.875165, 86.892988]

# northeastern corner
upper_right = [27.918111, 86.949540]

aoi = f'POLYGON(({lower_left[1]} {lower_left[0]}, {lower_left[1]} {upper_right[0]}, {upper_right[1]} {upper_right[0]}, {upper_right[1]} {lower_left[0]}, {lower_left[1]} {lower_left[0]}))'

In [10]:
# Set platform, start and end dates, flight direction, and processing level
opts = {'platform': asf.PLATFORM.SENTINEL1, 
        'start': '2021-08-01T00:00:00Z', 
        'end': '2021-09-01T00:00:00Z', 
        'flightDirection': 'A',
        'processingLevel': 'SLC',
        'frame': 84}

In [11]:
# search for scenes
results = asf.geo_search(intersectsWith=aoi, **opts)
print(f'{len(results)} scenes found')

3 scenes found


In [12]:
# Download scenes
%%time 

#Each scene is ~4 GB
results.download(path = '/tmp', session=user_pass_session)

UsageError: Line magic function `%%time` not found.


In [33]:
# Create a list of zipped SLCs. Assume there are no other files with .zip extension in tmp/SLC
from os import listdir

tmp = listdir('/tmp/SLC')

scenes = []
for file in tmp:
    if file[-4:]=='.zip':
        scenes.append(file[:-4])
        
print(scenes)

['S1A_IW_SLC__1SDV_20210830T121356_20210830T121423_039459_04A972_A57E', 'S1A_IW_SLC__1SDV_20210818T121355_20210818T121422_039284_04A383_54D5', 'S1A_IW_SLC__1SDV_20210806T121354_20210806T121421_039109_049D70_B0A0']


## Download precise orbitals 

In [29]:
os.chdir('/tmp/orbital')

In [14]:
%%bash

wget -nc https://raw.githubusercontent.com/isce-framework/isce2/main/contrib/stack/topsStack/fetchOrbit.py
chmod +x fetchOrbit.py

--2022-03-05 18:49:04--  https://raw.githubusercontent.com/isce-framework/isce2/main/contrib/stack/topsStack/fetchOrbit.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5054 (4.9K) [text/plain]
Saving to: ‘fetchOrbit.py’

     0K ....                                                  100% 51.8M=0s

2022-03-05 18:49:04 (51.8 MB/s) - ‘fetchOrbit.py’ saved [5054/5054]



In [15]:
for file in scenes:
    os.system(f'./fetchOrbit.py -i {file}')

Reference time:  2021-08-30 12:14:23
Satellite name:  S1A
Reference time:  2021-08-18 12:14:22
Satellite name:  S1A
Reference time:  2021-08-06 12:14:21
Satellite name:  S1A


### Also download calibration auxliary file
A calibration auxliary (AUX_CAL) file is used for antenna pattern correction to compensate the range phase offset of SAFE products with IPF verison 002.36 (mainly for images acquired before March 2015). 

In [39]:
os.chdir('/tmp/AUX')
!wget https://qc.sentinel1.groupcls.com/product/S1A/AUX_CAL/2014/09/08/S1A_AUX_CAL_V20140908T000000_G20190626T100201.SAFE.TGZ

--2022-03-05 19:24:40--  https://qc.sentinel1.groupcls.com/product/S1A/AUX_CAL/2014/09/08/S1A_AUX_CAL_V20140908T000000_G20190626T100201.SAFE.TGZ
Resolving qc.sentinel1.groupcls.com (qc.sentinel1.groupcls.com)... 20.74.8.22
Connecting to qc.sentinel1.groupcls.com (qc.sentinel1.groupcls.com)|20.74.8.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 494159 (483K)
Saving to: ‘S1A_AUX_CAL_V20140908T000000_G20190626T100201.SAFE.TGZ’

S1A_AUX_CAL_V201409 100%[===================>] 482.58K  --.-KB/s    in 0.04s   

2022-03-05 19:24:40 (10.6 MB/s) - ‘S1A_AUX_CAL_V20140908T000000_G20190626T100201.SAFE.TGZ’ saved [494159/494159]



## Specify or download digital elevation file

In [35]:
os.chdir('/tmp/DEM')

In [16]:
!which dem.py

/srv/conda/envs/isce2/lib/python3.9/site-packages/isce/applications/dem.py


In [41]:
print(f'lower left (southwest): {lower_left}, upper right (northwest): {upper_right}')

lower left (southwest): [27.875165, 86.892988], upper right (northwest): [27.918111, 86.94954]


In [37]:
# feed dem.py ingeter coordinates surrounding the aoi, using the following structure: lower lat, upper lat, left lon, right lon 

!dem.py -a stitch -b 27 28 86 87 -r -s 1 -c
!rm demLat*.dem demLat*.dem.xml demLat*.dem.vrt

This is the Open Source version of ISCE.
Some of the workflows depend on a separate licensed package.
To obtain the licensed package, please make a request for ISCE
through the website: https://download.jpl.nasa.gov/ops/request/index.cfm.
Alternatively, if you are a member, or can become a member of WinSAR
you may be able to obtain access to a version of the licensed sofware at
https://winsar.unavco.org/software/isce
curl -n  -L -c $HOME/.earthdatacookie -b $HOME/.earthdatacookie -k -f -O http://e4ftl01.cr.usgs.gov/MEASURES/SRTMGL1.003/2000.02.11/N27E086.SRTMGL1.hgt.zip
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   267  100   267    0     0   1091      0 --:--:-- --:--:-- --:--:--  1094
100 18.1M  100 18.1M    0     0   744k      0  0:00:25  0:00:25 --:--:-- 5529k
API open (R): ./demLat_N27_N28_Lon_E086_E087.dem
API close:  ./demLat_N27_N28_Lon_E086_E087.dem
Writing geo

## Stack processing 

In [46]:
os.chdir('/tmp')

In [22]:
# Look at options for stack processing

!stackSentinel.py -h

This is the Open Source version of ISCE.
Some of the workflows depend on a separate licensed package.
To obtain the licensed package, please make a request for ISCE
through the website: https://download.jpl.nasa.gov/ops/request/index.cfm.
Alternatively, if you are a member, or can become a member of WinSAR
you may be able to obtain access to a version of the licensed sofware at
https://winsar.unavco.org/software/isce
usage: stackSentinel.py [-h] [-H] -s SLC_DIRNAME -o ORBIT_DIRNAME -a
                        AUX_DIRNAME [-w WORK_DIR] -d DEM [-m REFERENCE_DATE]
                        [-c NUM_CONNECTIONS] [-n SWATH_NUM] [-b BBOX]
                        [-t TEXT_CMD] [-x EXCLUDE_DATES] [-i INCLUDE_DATES]
                        [--start_date STARTDATE] [--stop_date STOPDATE]
                        [-z AZIMUTHLOOKS] [-r RANGELOOKS] [-f FILTSTRENGTH]
                        [--snr_misreg_threshold SNRTHRESHOLD]
                        [-p POLARIZATION] [-C {geometry,NESD}]
              

In [47]:
!stackSentinel.py -s /tmp/SLC/ -o /tmp/orbital/ -a /tmp/AUX/ -d /tmp/DEM/demLat_N27_N28_Lon_E086_E087.dem.wgs84 -r 9 -z 2 -c 2 --bbox '27.8 28.00 86.80 87.00' -W interferogram

This is the Open Source version of ISCE.
Some of the workflows depend on a separate licensed package.
To obtain the licensed package, please make a request for ISCE
through the website: https://download.jpl.nasa.gov/ops/request/index.cfm.
Alternatively, if you are a member, or can become a member of WinSAR
you may be able to obtain access to a version of the licensed sofware at
https://winsar.unavco.org/software/isce
Number of SAFE files found: 3
Number of SAFE files to be used (cover BBOX): 3
*****************************************
Number of dates : 3
List of dates : 
['20210806', '20210818', '20210830']
date      south      north
20210806 26.30085 28.33621
20210818 26.301233 28.336365
20210830 26.300856 28.336529
*****************************************
The overlap region among all dates (based on the preview kml files):
 South   North   East  West 
26.301233 28.33621 86.205772 89.061569
*****************************************
All dates (3)
['20210806', '20210818', '20210830']



In [48]:
os.chdir('/tmp/run_files')

In [ ]:
%% time
!sh run_01_unpack_topo_reference

Completed Parsing the Configuration file
Functions to be executed:
['Function-1', 'Function-2']
Running: Sentinel1_TOPS
['--dirname', '/tmp/SLC/S1A_IW_SLC__1SDV_20210806T121354_20210806T121421_039109_049D70_B0A0.zip', '--swaths', '1 2 3', '--orbitdir', '/tmp/orbital', '--outdir', '/tmp/reference', '--auxdir', '/tmp/AUX', '--bbox', '27.8 28.00 86.80 87.00', '--pol', 'vv']
This is the Open Source version of ISCE.
Some of the workflows depend on a separate licensed package.
To obtain the licensed package, please make a request for ISCE
through the website: https://download.jpl.nasa.gov/ops/request/index.cfm.
Alternatively, if you are a member, or can become a member of WinSAR
you may be able to obtain access to a version of the licensed sofware at
https://winsar.unavco.org/software/isce
Input XML files:  ['/vsizip//tmp/SLC/S1A_IW_SLC__1SDV_20210806T121354_20210806T121421_039109_049D70_B0A0.zip/S1A_IW_SLC__1SDV_20210806T121354_20210806T121421_039109_049D70_B0A0.SAFE/annotation/s1a-iw1-slc-

In [ ]:
%%time
!sh run_02_unpack_secondary_slc

In [ ]:
%%time
!sh run_03_average_baseline

In [ ]:
%%time
!sh run_04_extract_burst_overlaps

In [ ]:
%%time
!sh run_05_overlap_geo2rdr

In [ ]:
%%time
!sh run_06_overlap_resample

In [ ]:
%%time
!sh run_07_pairs_misreg

In [ ]:
%%time
!sh run_08_timeseries_misreg

In [ ]:
%%time
!sh run_09_fullBurst_geo2rdr

In [ ]:
%%time
!sh run_10_fullBurst_resample

In [ ]:
%%time
!sh run_11_extract_stack_valid_region

In [ ]:
%%time
!sh run_12_merge_reference_secondary_slc

In [ ]:
%%time
!sh run_13_generate_burst_igram

In [ ]:
%%time
!sh run_14_merge_burst_igram

In [ ]:
%%time
!sh run_15_filter_coherence

In [ ]:
%%time
!sh run_16_unwrap